# Glass Classification

In [1]:
# Glass Classification project

![Alt text](https://www.missouriglass.com/wp-content/uploads/2021/05/types-of-glass-featured-image.jpeg)


In [3]:
#libraries
import pandas as pd
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import joblib
import warnings

warnings.filterwarnings("ignore")

In [4]:
#read data
df=pd.read_csv("/kaggle/input/glass/glass.csv")

In [5]:
#EDA
df.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


In [17]:
df.shape

(214, 10)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   RI      214 non-null    float64
 1   Na      214 non-null    float64
 2   Mg      214 non-null    float64
 3   Al      214 non-null    float64
 4   Si      214 non-null    float64
 5   K       214 non-null    float64
 6   Ca      214 non-null    float64
 7   Ba      214 non-null    float64
 8   Fe      214 non-null    float64
 9   Type    214 non-null    int64  
dtypes: float64(9), int64(1)
memory usage: 16.8 KB


In [8]:
df.corr()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
RI,1.000000,-0.191885,-0.122274,-0.407326,-0.542052,-0.289833,0.810403,-0.000386,0.143010,-0.164237
Na,-0.191885,1.000000,-0.273732,0.156794,-0.069809,-0.266087,-0.275442,0.326603,-0.241346,0.502898
Mg,-0.122274,-0.273732,1.000000,-0.481799,-0.165927,0.005396,-0.443750,-0.492262,0.083060,-0.744993
Al,-0.407326,0.156794,-0.481799,1.000000,-0.005524,0.325958,-0.259592,0.479404,-0.074402,0.598829
Si,-0.542052,-0.069809,-0.165927,-0.005524,1.000000,-0.193331,-0.208732,-0.102151,-0.094201,0.151565
K,-0.289833,-0.266087,0.005396,0.325958,-0.193331,1.000000,-0.317836,-0.042618,-0.007719,-0.010054
Ca,0.810403,-0.275442,-0.443750,-0.259592,-0.208732,-0.317836,1.000000,-0.112841,0.124968,0.000952
Ba,-0.000386,0.326603,-0.492262,0.479404,-0.102151,-0.042618,-0.112841,1.000000,-0.058692,0.575161
Fe,0.143010,-0.241346,0.083060,-0.074402,-0.094201,-0.007719,0.124968,-0.058692,1.000000,-0.188278
Type,-0.164237,0.502898,-0.744993,0.598829,0.151565,-0.010054,0.000952,0.575161,-0.188278,1.000000


In [10]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
RI,214.0,1.518365,0.003037,1.51115,1.516522,1.51768,1.519157,1.53393
Na,214.0,13.407850,0.816604,10.73000,12.907500,13.30000,13.825000,17.38000
Mg,214.0,2.684533,1.442408,0.00000,2.115000,3.48000,3.600000,4.49000
Al,214.0,1.444907,0.499270,0.29000,1.190000,1.36000,1.630000,3.50000
Si,214.0,72.650935,0.774546,69.81000,72.280000,72.79000,73.087500,75.41000
K,214.0,0.497056,0.652192,0.00000,0.122500,0.55500,0.610000,6.21000
Ca,214.0,8.956963,1.423153,5.43000,8.240000,8.60000,9.172500,16.19000
Ba,214.0,0.175047,0.497219,0.00000,0.000000,0.00000,0.000000,3.15000
Fe,214.0,0.057009,0.097439,0.00000,0.000000,0.00000,0.100000,0.51000
Type,214.0,2.780374,2.103739,1.00000,1.000000,2.00000,3.000000,7.00000


In [11]:
df.isnull().sum()

RI      0
Na      0
Mg      0
Al      0
Si      0
K       0
Ca      0
Ba      0
Fe      0
Type    0
dtype: int64

In [12]:
df["Type"].unique()

array([1, 2, 3, 5, 6, 7])

In [14]:
# we turn to object because they are nor oridinal
df["Type"]=df["Type"].astype("object")

In [15]:
#preprocesing
X=df.drop("Type",axis=1)
y=df["Type"]

In [16]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [21]:
#!pip install pycaret

In [22]:
#auto ml
from pycaret.classification import*
# setup data
setup(data=df,target="Type")

,Description,Value
0,Session id,550
1,Target,Type
2,Target type,Multiclass
3,Target mapping,"1: 0, 2: 1, 3: 2, 5: 3, 6: 4, 7: 5"
4,Original data shape,"(214, 10)"
5,Transformed data shape,"(214, 10)"
6,Transformed train set shape,"(149, 10)"
7,Transformed test set shape,"(65, 10)"
8,Numeric features,9
9,Preprocess,True


In [23]:
#compare models
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.8262,0.4750,0.8262,0.8246,0.8063,0.7614,0.7767,0.1340
catboost,CatBoost Classifier,0.8062,0.4769,0.8062,0.7953,0.7819,0.7330,0.7472,3.1210
lightgbm,Light Gradient Boosting Machine,0.8052,0.4796,0.8052,0.7864,0.7740,0.7284,0.7485,0.2540
rf,Random Forest Classifier,0.7867,0.4784,0.7867,0.7641,0.7636,0.7085,0.7184,0.1650
xgboost,Extreme Gradient Boosting,0.7390,0.4705,0.7390,0.7229,0.7135,0.6404,0.6558,0.0810
dt,Decision Tree Classifier,0.7129,0.4190,0.7129,0.7249,0.6960,0.6094,0.6270,0.0220
gbc,Gradient Boosting Classifier,0.6986,0.4618,0.6986,0.6516,0.6600,0.5803,0.5959,0.3440
lr,Logistic Regression,0.6567,0.4358,0.6567,0.5926,0.6150,0.5093,0.5229,0.6070
lda,Linear Discriminant Analysis,0.6567,0.4276,0.6567,0.6266,0.6265,0.5220,0.5367,0.0230
knn,K Neighbors Classifier,0.6390,0.4297,0.6390,0.6005,0.5986,0.4960,0.5164,0.0300


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='sqrt',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_samples_leaf=1,
                     min_samples_split=2, min_weight_fraction_leaf=0.0,
                     n_estimators=100, n_jobs=-1, oob_score=False,
                     random_state=550, verbose=0, warm_start=False)

In [24]:
#cros over best model
best_model = create_model('et')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8667,0.0000,0.8667,0.9000,0.8595,0.8077,0.8296
1,0.7333,0.0000,0.7333,0.8417,0.7009,0.6386,0.6932
2,0.7333,0.0000,0.7333,0.7492,0.7163,0.6296,0.6464
3,0.8667,0.0000,0.8667,0.8667,0.8667,0.8193,0.8193
4,0.8667,0.8943,0.8667,0.8111,0.8364,0.8171,0.8225
5,0.8667,0.9800,0.8667,0.8111,0.8364,0.8171,0.8225
6,0.8000,0.9886,0.8000,0.7714,0.7741,0.7256,0.7398
7,0.8000,0.9686,0.8000,0.8222,0.8054,0.7321,0.7365
8,0.8000,0.9190,0.8000,0.8048,0.7722,0.7321,0.7550


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [25]:
#save model
joblib.dump(best_model,"model.joblib")

['model.joblib']

In [26]:
#sources

-Zafer Acar Notes,-[https://thecleverprogrammer.com/]